# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-ewloxyhw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-ewloxyhw
  Resolved https://github.com/huggingface/accelerate.git to commit b451956fd69a135efc283aadaa478f0d33fcbe6a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-j5pc5e_1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-j5pc5e_1
  Resolved https://github.com/huggingface/transformers.git to commit 1d9743edc2030e8444d5bdffa910a3d6822bcf2d
  Installing build dependencies ... done
  Getting requirements to build whe

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:  97%|#########6| 9.62G/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

This function wraps the call to *model.generate*

In [8]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [25]:
sp_nl2sql = """
  ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name TEXT,
    price DECIMAL
);

CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);


    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [26]:
sp_nl2sql = sp_nl2sql.format(question="What are the names of all customers who placed an order in January?")
print(sp_nl2sql)



  ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name TEXT,
    price DECIMAL
);

CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_i

In [27]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [28]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [29]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.name FROM Customers c JOIN Orders o ON c.customer_id = o.customer_id WHERE date_part('year', o.order_date) = date_part('year', CURRENT_DATE) AND date_part('month', o.order_date) = 1;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [30]:
sp_nl2sql2 = """
### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name TEXT,
    price DECIMAL
);

CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);

  ### Response

    Question1: List all product names and their prices.
    ```sql
    SELECT name, price FROM Products;

    Question2: Get the number of orders placed by each customer
    ```sql
    SELECT c.name, COUNT(o.order_id) AS order_count
    FROM Customers c
    JOIN Orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;

      Based on your instructions, here is the SQL query I have generated to answer the question
      `{question}`:
      ```sql3
      """


In [31]:
sp_nl2sql2 = sp_nl2sql2.format(question="What are the names of all customers who placed an order in January?")
(print(sp_nl2sql2))


### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name TEXT,
    price DECIMAL
);

CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)


In [32]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [33]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.name
       FROM Customers c
       JOIN Orders o ON c.customer_id = o.customer_id
       WHERE date_part('year', o.order_date) = date_part('year', '2020-01-01')
       AND date_part('month', o.order_date) = date_part('month', '2020-01-01')
       AND date_part('day', o.order_date) BETWEEN date_part('day', '2020-01-01') AND date_part('day', '2020-01-31');


#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [34]:
sp_nl2sql3b = """
### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name TEXT,
    price DECIMAL
);

CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);


    ### Samples

    Question1: List all customer names.
    ```sql
    SELECT name FROM Customers;

    Question2: Show names of all products that were ordered more than once
    ```sql
    SELECT p.name
    FROM Products p
    JOIN OrderItems oi ON p.product_id = oi.product_id
    GROUP BY p.name
    HAVING SUM(oi.quantity) > 1;

    Question3:Retrieve the total number of orders per customer
    ```sql
    SELECT c.name, COUNT(o.order_id) AS order_count
    FROM Customers c
    JOIN Orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;


    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [35]:
sp_nl2sql3 = sp_nl2sql3b.format(question="What are the names of all customers who placed an order in January?")
print (sp_nl2sql3)


### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name TEXT,
    price DECIMAL
);

CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)

In [36]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [37]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.name
    FROM Customers c
    JOIN Orders o ON c.customer_id = o.customer_id
    WHERE date_part('year', o.order_date) = date_part('year', CURRENT_DATE)
    AND date_part('month', o.order_date) = 1;


#Now the question in spanish.


In [38]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Cuáles son los nombres de todos los clientes que hicieron un pedido en enero")
print (sp_nl2sql3)


### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name TEXT,
    price DECIMAL
);

CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)

In [39]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [40]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.name, COUNT(o.order_id) AS order_count
    FROM Customers c
    JOIN Orders o ON c.customer_id = o.customer_id
    WHERE o.order_date BETWEEN '2020-01-01' AND '2020-01-31'
    GROUP BY c.name
    HAVING COUNT(o.order_id) > 0;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

## Version 1

In [56]:
sp_nl2sql_v1 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

  ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [57]:
sp_nl2sql_v1_ = sp_nl2sql_v1.format(question="What are the names of customers who have placed more than 3 orders and spent over 500 in total")
print (sp_nl2sql_v1_)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

  ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `What are the names of customers who have placed more than 3 orders and spent over 500 in total`:
    ```sql3
    


In [58]:
input_sentences = tokenizer(sp_nl2sql_v1_, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [59]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.name, COUNT(o.order_id) AS order_count, SUM(o.total_amount) AS total_spent FROM Customers c JOIN Orders o ON c.customer_id = o.customer_id GROUP BY c.name HAVING COUNT(o.order_id) > 3 AND SUM(o.total_amount) > 500;


## Version 2

In [47]:
sp_nl2sql_v2 = """
### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

    ### Response

    Question1: List all customer names.
    ```sql
    SELECT name FROM Customers;

    Question2:Retrieve the total number of orders per customer
    ```sql
    SELECT c.name, COUNT(o.order_id) AS order_count
    FROM Customers c
    JOIN Orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;



    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [48]:
sp_nl2sql_v2_ = sp_nl2sql_v2.format(question="What are the names of customers who have placed more than 3 orders and spent over 500 in total")
print (sp_nl2sql_v2_)


### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

    ### Response

    Question1: List all customer names.
    ```sql
    SELECT name FROM Customers;

    Question2:Retrieve the total number of orders per customer
    ```sql
    SELECT c.name, COUNT(o.order_id) AS order_count
    FROM Customers c
    JOIN Orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;



    Bas

In [50]:
input_sentences = tokenizer(sp_nl2sql_v2_, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [51]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.name, COUNT(o.order_id) AS order_count, SUM(o.total_amount) AS total_spent
     FROM Customers c
     JOIN Orders o ON c.customer_id = o.customer_id
     GROUP BY c.name
     HAVING COUNT(o.order_id) > 3 AND SUM(o.total_amount) > 500;


## Version 3

In [52]:
sp_nl2sql_v3 = """
   ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

     ### Response

    Question1: List all customer names.
    ```sql
    SELECT name FROM Customers;

    Question2:Retrieve the total number of orders per customer
    ```sql
    SELECT c.name, COUNT(o.order_id) AS order_count
    FROM Customers c
    JOIN Orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;

    Question 3: Find customers who have spent more than 500 in total in the year 2023.
    ```sql
    SELECT Customers.name
    FROM Customers
    JOIN Orders ON Customers.customer_id = Orders.customer_id
    WHERE Orders.order_date BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY Customers.name
    HAVING SUM(Orders.total_amount) > 500;



    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [53]:
sp_nl2sql_v3_ = sp_nl2sql_v3.format(question="What are the names of customers who have placed more than 3 orders and spent over 500 in total")
print (sp_nl2sql_v2_)


### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name TEXT,
    email TEXT
);
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

    ### Response

    Question1: List all customer names.
    ```sql
    SELECT name FROM Customers;

    Question2:Retrieve the total number of orders per customer
    ```sql
    SELECT c.name, COUNT(o.order_id) AS order_count
    FROM Customers c
    JOIN Orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;



    Bas

In [54]:
input_sentences = tokenizer(sp_nl2sql_v3_, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [55]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT Customers.name
    FROM Customers
    JOINOrders ON Customers.customer_id =Orders.customer_id
    WHEREOrders.order_date BETWEEN '2023-01-01' AND '2023-12-31'
    GROUP BY Customers.name
    HAVING COUNT(Orders.order_id) > 3 AND SUM(Orders.total_amount) > 500;


# SQL Prompt Engineering Evaluation Report

## Task Description
The goal was to evaluate the performance of three prompt variations designed to convert a natural language question into a valid SQL query.  
The question used in all prompts was:

> **"What are the names of customers who have placed more than 3 orders and spent over 500 in total?"**

The database schema included:
- `Customers(customer_id, name, email)`
- `Orders(order_id, customer_id, order_date, total_amount)`

---

## Prompt Versions

###**Version 1**

Correct and concise

Matches the intent of the question accurately.

No hallucinations or irrelevant conditions.

### **Version 2**

Also correct

Nearly identical to Version 1.

### **Version 3**

Issues found:

Formatting errors: no spaces between SQL keywords (e.g., JOINOrders, WHEREOrders)

Added a filter for 2023 not mentioned in the question → hallucination

Resulting query does not match the question's intent